In [64]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [65]:
print("hello world")

hello world


# Importing Libraries

In [66]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [67]:
from PIL import Image  # Importing the Image module
import numpy as np
import matplotlib.pyplot as plt

# Hyperparameters 

In [68]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1

# Dynamically set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

GRAYSCALE = False

Using device: cuda


In [69]:
train_size = int(10000*.8)
validation_size = int(10000*.1)
test_size = int(10000*.1)

In [70]:
# import argparse
# import os

# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# from tqdm import tqdm

# from net.models import LeNet
# from net.quantization import apply_weight_sharing
# import util

# os.makedirs('saves', exist_ok=True)

# # Training settings
# parser = argparse.ArgumentParser(description='PyTorch MNIST pruning from deep compression paper')
# parser.add_argument('--batch-size', type=int, default=50, metavar='N',
#                     help='input batch size for training (default: 50)')
# parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
#                     help='input batch size for testing (default: 1000)')
# parser.add_argument('--epochs', type=int, default=100, metavar='N',
#                     help='number of epochs to train (default: 100)')
# parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                     help='learning rate (default: 0.01)')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='disables CUDA training')
# parser.add_argument('--seed', type=int, default=42, metavar='S',
#                     help='random seed (default: 42)')
# parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                     help='how many batches to wait before logging training status')
# parser.add_argument('--log', type=str, default='log.txt',
#                     help='log file name')
# parser.add_argument('--sensitivity', type=float, default=2,
#                     help="sensitivity value that is multiplied to layer's std in order to get threshold value")
# args = parser.parse_args()

# # Control Seed
# torch.manual_seed(args.seed)

# # Select Device
# use_cuda = not args.no_cuda and torch.cuda.is_available()
# device = torch.device("cuda" if use_cuda else 'cpu')
# if use_cuda:
#     print("Using CUDA!")
#     torch.cuda.manual_seed(args.seed)
# else:
#     print('Not using CUDA!!!')

# # Loader
# kwargs = {'num_workers': 5, 'pin_memory': True} if use_cuda else {}
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=False, **kwargs)


# # Define which model to use
# model = LeNet(mask=True).to(device)

# print(model)
# util.print_model_parameters(model)

# # NOTE : `weight_decay` term denotes L2 regularization loss term
# optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.0001)
# initial_optimizer_state_dict = optimizer.state_dict()

# def train(epochs):
#     model.train()
#     for epoch in range(epochs):
#         pbar = tqdm(enumerate(train_loader), total=len(train_loader))
#         for batch_idx, (data, target) in pbar:
#             data, target = data.to(device), target.to(device)
#             optimizer.zero_grad()
#             output = model(data)
#             loss = F.nll_loss(output, target)
#             loss.backward()

#             # zero-out all the gradients corresponding to the pruned connections
#             for name, p in model.named_parameters():
#                 if 'mask' in name:
#                     continue
#                 tensor = p.data.cpu().numpy()
#                 grad_tensor = p.grad.data.cpu().numpy()
#                 grad_tensor = np.where(tensor==0, 0, grad_tensor)
#                 p.grad.data = torch.from_numpy(grad_tensor).to(device)

#             optimizer.step()
#             if batch_idx % args.log_interval == 0:
#                 done = batch_idx * len(data)
#                 percentage = 100. * batch_idx / len(train_loader)
#                 pbar.set_description(f'Train Epoch: {epoch} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')


# def test():
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
#             pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
#             correct += pred.eq(target.data.view_as(pred)).sum().item()

#         test_loss /= len(test_loader.dataset)
#         accuracy = 100. * correct / len(test_loader.dataset)
#         print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')
#     return accuracy


# # Initial training
# print("--- Initial training ---")
# train(args.epochs)
# accuracy = test()
# util.log(args.log, f"initial_accuracy {accuracy}")
# torch.save(model, f"saves/initial_model.ptmodel")
# print("--- Before pruning ---")
# util.print_nonzeros(model)

# # Pruning
# model.prune_by_std(args.sensitivity)
# accuracy = test()
# util.log(args.log, f"accuracy_after_pruning {accuracy}")
# print("--- After pruning ---")
# util.print_nonzeros(model)

# # Retrain
# print("--- Retraining ---")
# optimizer.load_state_dict(initial_optimizer_state_dict) # Reset the optimizer
# train(args.epochs)
# torch.save(model, f"saves/model_after_retraining.ptmodel")
# accuracy = test()
# util.log(args.log, f"accuracy_after_retraining {accuracy}")

# print("--- After Retraining ---")
# util.print_nonzeros(model)

# Model development and Fine-tuning

In [71]:
# import torchvision.models as models

# model = models.resnet18()


# model_path = "/kaggle/input/resnet18_baseline_celeba/pytorch/default/1/fine_tuned_best_ever_resnet18"


In [72]:
import torchvision.models as models

model = models.resnet18()


model_path = "/kaggle/input/resnet18_baseline_celeba/pytorch/default/1/fine_tuned_best_ever_resnet18"

# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer of the model
num_classes = 2 # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# # Freeze all the pre-trained layers
# for param in model.fc.parameters():
#     param.requires_grad = True

for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_30/460664334.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [73]:
# # Modify the last layer of the model
# num_classes = 2 # replace with the number of classes in your dataset
# model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# # Freeze all the pre-trained layers
# for param in model.fc.parameters():
#     param.requires_grad = True

In [74]:
# model.load_state_dict(torch.load(model_path))

# Dataset Loaders
The target label is attractiveness and protected attribute is gender.

In [75]:
import os

import numpy as np
import pandas as pd

import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

from PIL import Image

In [76]:
#The target label is attractiveness and protected attribute is gender.

data_root = "/kaggle/input/celeba-dataset/"
label_attr = "Attractive"
protected_attr = "Male"

In [77]:
pd.read_csv(data_root + 'list_attr_celeba.csv').columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [78]:
df1 = pd.read_csv(data_root + 'list_attr_celeba.csv', usecols=[label_attr, protected_attr])
df1.head()

,Attractive,Male
0,1,-1
1,-1,-1
2,-1,1
3,1,-1
4,1,-1


In [79]:
df1.loc[df1[label_attr] == -1, label_attr] = 0
df1.loc[df1[protected_attr] == -1, protected_attr] = 0
df1.head()

,Attractive,Male
0,1,0
1,0,0
2,0,1
3,1,0
4,1,0


In [80]:
df2 = pd.read_csv(data_root + 'list_eval_partition.csv')

df2.columns = ['Filename', 'Partition']
image_series = df2['Filename']
df1 = df1.set_index(image_series)
df2 = df2.set_index('Filename')
df2.head()

,Partition
Filename,
000001.jpg,0
000002.jpg,0
000003.jpg,0
000004.jpg,0
000005.jpg,0


In [81]:
pd.concat([df1, df2], axis=1)

,Attractive,Male,Partition
Filename,,,
000001.jpg,1,0,0
000002.jpg,0,0,0
000003.jpg,0,1,0
000004.jpg,1,0,0
000005.jpg,1,0,0
...,...,...,...
202595.jpg,1,0,2
202596.jpg,0,1,2
202597.jpg,0,1,2


In [82]:
# Hyper-params
BATCH_SIZE = 256

In [83]:
class CelebaDataset(Dataset):
    
    def __init__(self, csv_path, img_dir, transform=None, label_attr = 'Attractive', protected_attr = 'Male'):
        
        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.index.values
        self.y = df[label_attr].values
        self.p = df[protected_attr].values
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                      self.img_names[index]))

        if self.transform is not None:
            img = self.transform(img)
            
        label = self.y[index]
        protected = self.p[index]
        return img, label, protected
        
    def __len__(self):
        return self.y.shape[0]


In [84]:
other_data_root = '/kaggle/working/' #for writing purposes

In [85]:
def prepare_dataset(label_attr = 'Attractive', protected_attr = 'Male'):

    print('Using label attribute:', label_attr)

    df1 = pd.read_csv(data_root + 'list_attr_celeba.csv',usecols=[label_attr, protected_attr])
    # Make 0 (female) & 1 (male) labels instead of -1 & 1
    df1.loc[df1[label_attr] == -1, label_attr] = 0
    df1.loc[df1[protected_attr] == -1, protected_attr] = 0
    
#     print(f"df1\n{df1}")

    df2 = pd.read_csv(data_root + 'list_eval_partition.csv')
    df2.columns = ['image_id', 'partition']
    image_series = df2['image_id']
    df2 = df2.set_index('image_id')
#     print(f"df2\n{df2}")

#     print("\n\n")
#     print(f"df2 shape {df2.shape} and df1 shape {df1.shape}")

    df1 = df1.set_index(image_series)

#     df3 = df1.merge(df2, left_index=True, right_index=True)
    df3 = pd.concat([df1,df2],axis=1)
#     print("\n\n\n\n")
#     print(df1.index)  
#     print(df2.index)
#     print("\n\n\n\n")
    
#     print(f"alt df3 \n{ df1.merge(df2, left_index=True, right_index=True)}")
#     print(f"df3\n{df3}")

    
    
    
    df3.to_csv(other_data_root + 'celeba-attractive-partitions.csv')
#     print(df3)
#     print(pd.read_csv(new_data_root+'celeba-attractive-partitions.csv'))
    df4 = pd.read_csv(other_data_root + 'celeba-attractive-partitions.csv', index_col=0)

    df4.loc[df4['partition'] == 0].to_csv(other_data_root + 'celeba-attractive-train.csv')
    df4.loc[df4['partition'] == 1].to_csv(other_data_root + 'celeba-attractive-valid.csv')
    df4.loc[df4['partition'] == 2].to_csv(other_data_root + 'celeba-attractive-test.csv')
    
#     print(f"Train samples: {df4.loc[df4['partition'] == 0].shape[0]}")
#     print(f"Valid samples: {df4.loc[df4['partition'] == 1].shape[0]}")
#     print(f"Test samples: {df4.loc[df4['partition'] == 2].shape[0]}")

    
def get_loaders():

    prepare_dataset(label_attr, protected_attr)
    
    # Note that transforms.ToTensor()
    # already divides pixels by 255. internally

    custom_transform = transforms.Compose([transforms.CenterCrop((178, 178)),
                                           transforms.Resize((128, 128)),
                                           transforms.ToTensor()])

    train_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-train.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    valid_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-valid.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    test_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-test.csv',
                                 img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                 transform=custom_transform,
                                 label_attr=label_attr,
                                 protected_attr = protected_attr)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=4)

    valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=4)
    
    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=4)

    return train_loader, valid_loader, test_loader


## main.py running 

### Few libraries

In [86]:
import os

import numpy as np
import pandas as pd

import torch

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

from PIL import Image

In [87]:
# from datasets_celebA import get_loaders
# from resnet import resnet18

# from utils import *


### configuration

In [88]:
config = {}
# # Option to choose between different algorithms
# config['ALGORITHM'] = 'FAIR_ALM'  # 'FAIR_ALM' or 'L2_PENALTY' or 'NO_CONSTRAINTS' 
# config['CONSTRAINT'] = 'DEO' # DEO, DDP, PPV 
# config['LAM0_PRIOR'] = 0.
# config['LAM1_PRIOR'] = 0.
# config['LAM2_PRIOR'] = 0.
# config['ETA_INIT'] = 20
# config['ETA_BETA'] = 1.01
# config['SAVE_CKPT'] = False
# config['DEBUG'] = False

# # storing checkpoints 
# config['file_name'] = '/kaggle/working/'

### hyperparameters and architecture

In [89]:
# Hyperparameters
config['LR'] = 0.0001
config['NUM_EPOCHS'] = 25
config['NUM_INNER'] = 5

In [90]:
# Architecture
NUM_FEATURES = 128*128
NUM_CLASSES = 2
GRAYSCALE = False

config['OPTIMIZER_'] = 'SGD'
config['MODEL_'] = 'resnet18'
config['SHUFFLE_'] = True

### functions for training and pre-processing

In [91]:
def save_everything(epoch, net, optimizer, train_acc, val_acc):
    # Save checkpoint.
    state = {
        'epoch': epoch,
        'net': net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'train_acc': train_acc,
        'val_acc': val_acc
    }
    if not os.path.isdir(config['file_name']):
        os.mkdir(config['file_name'])
    torch.save(state, config['file_name'] + '/ckpt_' + str(epoch) + '.t7')

In [92]:
train_loader, valid_loader, test_loader = get_loaders()
#here model is already defined

Using label attribute: Attractive


In [93]:
# #idk what this does
# #### DATA PARALLEL START ####
# if torch.cuda.device_count() > 1:
# 	print("Using", torch.cuda.device_count(), "GPUs")
# 	model = nn.DataParallel(model)
# #### DATA PARALLEL END ####


In [94]:
# model = model.cuda()
# criterion = torch.nn.CrossEntropyLoss(reduce=False)
# optimizer = torch.optim.SGD(model.parameters(), lr=config['LR'])

In [95]:
# lam0 = config['LAM0_PRIOR'] 
# lam1 = config['LAM1_PRIOR']
# lam2 = config['LAM2_PRIOR'] 
# eta = config['ETA_INIT']

In [96]:
pd.DataFrame([config])

,LR,NUM_EPOCHS,NUM_INNER,OPTIMIZER_,MODEL_,SHUFFLE_
0,0.0001,25,5,SGD,resnet18,True


## Fine tuning

In [97]:
import time
import os

In [98]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Initialize scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

# Early stopping variables
best_val_accuracy = 0.0
patience = 5  # Stop if no improvement after 5 epochs
epochs_without_improvement = 0

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [99]:
config = {}
# Option to choose between different algorithms
config['ALGORITHM'] = 'FAIR_ALM'  # 'FAIR_ALM' or 'L2_PENALTY' or 'NO_CONSTRAINTS' 
config['CONSTRAINT'] = 'DEO' # DEO, DDP, PPV 
config['LAM0_PRIOR'] = 0.
config['LAM1_PRIOR'] = 0.
config['LAM2_PRIOR'] = 0.
config['ETA_INIT'] = 20
config['ETA_BETA'] = 1.01
config['SAVE_CKPT'] = False
config['DEBUG'] = False

# Architecture
NUM_FEATURES = 128*128
NUM_CLASSES = 2
GRAYSCALE = False
# Hyperparameters
config['LR'] = 0.001
config['NUM_EPOCHS'] = 25
config['NUM_INNER'] = 5
config['OPTIMIZER_'] = 'SGD'
config['MODEL_'] = 'resnet18'
config['SHUFFLE_'] = True

model = model.cuda()
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.SGD(model.parameters(), lr=config['LR'])

#idk what this does
#### DATA PARALLEL START ####
if torch.cuda.device_count() > 1:
	print("Using", torch.cuda.device_count(), "GPUs")
	model = nn.DataParallel(model)
#### DATA PARALLEL END ####


lam0 = config['LAM0_PRIOR'] 
lam1 = config['LAM1_PRIOR']
lam2 = config['LAM2_PRIOR'] 
eta = config['ETA_INIT']


# storing checkpoints 
config['file_name'] = '/kaggle/working/'

start_time = time.time()

for epoch in range(config['NUM_EPOCHS']):
    eta *= config['ETA_BETA']  # Update dual parameter
    model.train()

    for batch_idx, (features, targets, protected) in enumerate(train_loader):
        if config['DEBUG'] and batch_idx > 1:
            break

        features, targets, protected = features.cuda(), targets.cuda(), protected.cuda()
        
        if config['ALGORITHM'] == 'FAIR_ALM':
            for _ in range(config['NUM_INNER']):
                # preprocess = transforms.Compose([
                #     transforms.Resize((224, 224)),
                #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                # ])
                input_tensor = torch.stack([image for image in features])
                logits = model(input_tensor)

                loss_all = criterion(logits, targets)
                loss_t0_s0 = loss_all[(targets == 0) & (protected == 0)].mean()
                loss_t0_s1 = loss_all[(targets == 0) & (protected == 1)].mean()
                loss_t1_s0 = loss_all[(targets == 1) & (protected == 0)].mean()
                loss_t1_s1 = loss_all[(targets == 1) & (protected == 1)].mean()
                train_loss = loss_all.mean()
                penalty_loss = loss_t1_s0 - loss_t1_s1

                optimizer.zero_grad()
                loss = (
                    loss_all.mean() 
                    + (eta / 4 + (lam0 - lam1) / 2) * loss_t1_s0 
                    + (eta / 4 + (lam1 - lam0) / 2) * loss_t1_s1
                )
                loss.backward()
                optimizer.step()

            lam0 = 0.5 * (lam0 - lam1) + 0.5 * eta * (loss_t1_s0.item() - loss_t1_s1.item())
            lam1 = 0.5 * (lam1 - lam0) + 0.5 * eta * (loss_t1_s1.item() - loss_t1_s0.item())

        if batch_idx % 50 == 0:
            print(f"Epoch: {epoch+1}/{config['NUM_EPOCHS']} | Batch: {batch_idx}/{len(train_loader)} "
                  f"| Train Loss: {train_loss:.4f} | Penalty Loss: {penalty_loss:.4f}")
            print(f"eta: {eta:.3f} | lam0: {lam0:.3f} | lam1: {lam1:.3f}")

    # Validation step
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for val_features, val_targets, _ in valid_loader:
            val_features, val_targets = val_features.cuda(), val_targets.cuda()
            val_logits = model(val_features)
            val_preds = torch.argmax(val_logits, dim=1)
            val_correct += (val_preds == val_targets).sum().item()
            val_total += val_targets.size(0)

    val_accuracy = val_correct / val_total
    print(f"Epoch {epoch+1}: Validation Accuracy: {val_accuracy:.4f}")
    
    # Scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_without_improvement = 0
        # Optionally save the best model
        if config['SAVE_CKPT']:
            torch.save(model.state_dict(), config['file_name'] + 'best_model.pth')
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"No improvement for {patience} epochs. Early stopping...")
        break

print(f"Training completed in {time.time() - start_time:.2f} seconds.")


Using 2 GPUs


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Epoch: 1/25 | Batch: 0/636 | Train Loss: 0.0241 | Penalty Loss: -0.0701
eta: 20.200 | lam0: -0.708 | lam1: 1.062
Epoch: 1/25 | Batch: 50/636 | Train Loss: 0.0911 | Penalty Loss: 0.0006
eta: 20.200 | lam0: -2.242 | lam1: 2.230
Epoch: 1/25 | Batch: 100/636 | Train Loss: 0.0534 | Penalty Loss: -0.0626
eta: 20.200 | lam0: -1.179 | lam1: 1.504
Epoch: 1/25 | Batch: 150/636 | Train Loss: 0.0284 | Penalty Loss: 0.0036
eta: 20.200 | lam0: -2.519 | lam1: 2.507
Epoch: 1/25 | Batch: 200/636 | Train Loss: 0.0276 | Penalty Loss: 0.0014
eta: 20.200 | lam0: -1.944 | lam1: 1.933
Epoch: 1/25 | Batch: 250/636 | Train Loss: 0.0580 | Penalty Loss: -0.0009
eta: 20.200 | lam0: -1.683 | lam1: 1.681
Epoch: 1/25 | Batch: 300/636 | Train Loss: 0.1260 | Penalty Loss: 0.0000
eta: 20.200 | lam0: -2.698 | lam1: 2.697
Epoch: 1/25 | Batch: 350/636 | Train Loss: 0.0844 | Penalty Loss: 0.0008
eta: 20.200 | lam0: -1.325 | lam1: 1.322
Epoch: 1/25 | Batch: 400/636 | Train Loss: 0.0680 | Penalty Loss: -0.0031
eta: 20.200 | 

In [119]:
def test_model(model_in_fn, test_loader, gpu_yes = True):
    model_in_fn.eval()  # Set model to evaluation mode
    if gpu_yes:
        device = 'cuda'
        model_in_fn.to(device)
    else:
        device='cpu'
        model_in_fn.to('cpu')
    # Initialize counters
    correct, total = 0, 0
    male_correct, female_correct = 0, 0
    male_total, female_total = 0, 0
    
    # Initialize confusion matrix components for males and females
    male_tp, male_fp, male_tn, male_fn = 0, 0, 0, 0
    female_tp, female_fp, female_tn, female_fn = 0, 0, 0, 0
    all_ones, all_zeros = 0, 0
    
    with torch.no_grad():
        for images, labels, genders in test_loader:
            images, labels, genders = images.to(device), labels.to(device), genders.to(device)
            outputs = model_in_fn(images)
            _, predicted = torch.max(outputs, 1)
            
            # Update overall accuracy
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            # Count total positive and negative cases
            all_ones += (labels == 1).sum().item()
            all_zeros += (labels == 0).sum().item()
            
            # Separate male and female predictions
            male_mask = (genders == 1)
            female_mask = (genders == 0)
            
            # Update accuracy counts
            male_correct += (predicted[male_mask] == labels[male_mask]).sum().item()
            female_correct += (predicted[female_mask] == labels[female_mask]).sum().item()
            male_total += male_mask.sum().item()
            female_total += female_mask.sum().item()
            
            # Update confusion matrix components for males
            male_tp += ((predicted[male_mask] == 1) & (labels[male_mask] == 1)).sum().item()
            male_fp += ((predicted[male_mask] == 1) & (labels[male_mask] == 0)).sum().item()
            male_tn += ((predicted[male_mask] == 0) & (labels[male_mask] == 0)).sum().item()
            male_fn += ((predicted[male_mask] == 0) & (labels[male_mask] == 1)).sum().item()
            
            # Update confusion matrix components for females
            female_tp += ((predicted[female_mask] == 1) & (labels[female_mask] == 1)).sum().item()
            female_fp += ((predicted[female_mask] == 1) & (labels[female_mask] == 0)).sum().item()
            female_tn += ((predicted[female_mask] == 0) & (labels[female_mask] == 0)).sum().item()
            female_fn += ((predicted[female_mask] == 0) & (labels[female_mask] == 1)).sum().item()
    
    # Calculate metrics
    overall_accuracy = correct / total
    male_accuracy = male_correct / male_total if male_total > 0 else 0
    female_accuracy = female_correct / female_total if female_total > 0 else 0
    
    # Calculate TPR and FPR correctly
    male_tpr = male_tp / (male_tp + male_fn) if (male_tp + male_fn) > 0 else 0
    female_tpr = female_tp / (female_tp + female_fn) if (female_tp + female_fn) > 0 else 0
    male_fpr = male_fp / (male_fp + male_tn) if (male_fp + male_tn) > 0 else 0
    female_fpr = female_fp / (female_fp + female_tn) if (female_fp + female_tn) > 0 else 0
    
    # Fairness metrics
    DEO = abs(male_tpr - female_tpr)
    FPR_diff = abs(male_fpr - female_fpr)
    
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    print(f"Male Accuracy: {male_accuracy:.4f}")
    print(f"Female Accuracy: {female_accuracy:.2f}")
    print(f"Male TPR: {male_tpr:.2f}")
    print(f"Female TPR: {female_tpr:.2f}")
    print(f"Male FPR: {male_fpr:.2f}")
    print(f"Female FPR: {female_fpr:.2f}")
    print(f"Demographic Equal Opportunity (DEO): {DEO:.2f}")
    print(f"False Positive Rate Difference (FPR): {FPR_diff:.2f}")
    
    return {
        "overall_accuracy": overall_accuracy,
        "male_accuracy": male_accuracy,
        "female_accuracy": female_accuracy,
        "male_tpr": male_tpr,
        "female_tpr": female_tpr,
        "male_fpr": male_fpr,
        "female_fpr": female_fpr,
        "DEO": DEO,
        "FPR_difference": FPR_diff,
        "Attractive yes in test": all_ones/total,
        "Attractive no in test": all_zeros/total,
        "Batch Size": BATCH_SIZE
    }, model_in_fn

In [120]:
test_model(model,test_loader)

Overall Accuracy: 0.7962
Male Accuracy: 0.7983
Female Accuracy: 0.79
Male TPR: 0.74
Female TPR: 0.92
Male FPR: 0.18
Female FPR: 0.43
Demographic Equal Opportunity (DEO): 0.18
False Positive Rate Difference (FPR): 0.25


({'overall_accuracy': 0.7962128043282236,
  'male_accuracy': 0.7983149708360338,
  'female_accuracy': 0.7948885441332572,
  'male_tpr': 0.7366771159874608,
  'female_tpr': 0.9154559118236473,
  'male_fpr': 0.18134804344078606,
  'female_fpr': 0.4309171944639925,
  'DEO': 0.1787787958361865,
  'FPR_difference': 0.24956915102320645,
  'Attractive yes in test': 0.49584209998998097,
  'Attractive no in test': 0.504157900010019,
  'Batch Size': 256},
 DataParallel(
   (module): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): BasicBlock(
         (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

In [121]:
torch.save(model, "/kaggle/working/fairALM_finetuned_celebA")

## Testing

In [103]:
def get_res_dict():
    res = {'acc': None,
           'ddp': None,
           'ppv': None,
           'fpr': None,
           'fnr': None,
           'tn_s0': None,
           'tn_s1': None,
           'fp_s0': None,
           'fp_s1': None,
           'fn_s0': None,
           'fn_s1': None,
           'tp_s0': None,
           'tp_s1': None
    }
    return res

In [104]:
def compute_accuracy(config, model, data_loader):
    correct_pred, num_examples = 0, 0

    num_protected0, num_protected1 = 0, 0
    num_correct_pred_protected0, num_correct_pred_protected1 = 0, 0

    num_pred1_protected0, num_pred1_protected1 = 0, 0
    num_targets0_protected0, num_targets0_protected1 = 0, 0
    num_targets1_protected0, num_targets1_protected1 = 0, 0
    
    num_pred0_targets0_protected0, num_pred0_targets0_protected1 = 0, 0
    num_pred1_targets0_protected0, num_pred1_targets0_protected1 = 0, 0
    num_pred0_targets1_protected0, num_pred0_targets1_protected1 = 0, 0
    num_pred1_targets1_protected0, num_pred1_targets1_protected1 = 0, 0

    for i, (features, targets, protected) in enumerate(data_loader):
        if config['DEBUG'] and i > 1:
            break
        features = features.cuda()
        targets = targets.cuda()
        protected = protected.cuda()
#         logits, probas = model(features)
        
        preprocess = transforms.Compose([
                    transforms.Resize((224, 224)),  # Resizing to 224x224
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

                # Apply preprocessing to each image in the batch
        input_tensor = torch.stack([preprocess(image) for image in features])

                # Forward pass to get logits
        logits = model(input_tensor)
        probas = torch.nn.functional.softmax(logits, dim=1)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += 1.*targets.size(0) 
        correct_pred += (predicted_labels == targets).float().sum() # CHANGED

        # For DDP metric
        num_protected0 += (protected == 0).float().sum()
        num_protected1 += (protected == 1).float().sum()
        num_correct_pred_protected0 += ((predicted_labels == targets) & (protected == 0)).float().sum()
        num_correct_pred_protected1 += ((predicted_labels == targets) & (protected == 1)).float().sum()

        # For PPV metric
        num_pred1_protected0 += ((predicted_labels == 1) & (protected == 0)).float().sum()
        num_pred1_protected1 += ((predicted_labels == 1) & (protected == 1)).float().sum()
        
        # For FPR metric
        num_targets0_protected0 += ((targets == 0) & (protected == 0)).float().sum()
        num_targets0_protected1 += ((targets == 0) & (protected == 1)).float().sum()
        
        # For FNR metric
        num_targets1_protected0 += ((targets == 1) & (protected == 0)).float().sum()
        num_targets1_protected1 += ((targets == 1) & (protected == 1)).float().sum()

        num_pred0_targets0_protected0 += ((predicted_labels == 0) & (targets == 0) & (protected == 0)).float().sum()
        num_pred0_targets0_protected1 += ((predicted_labels == 0) & (targets == 0) & (protected == 1)).float().sum()
        num_pred1_targets0_protected0 += ((predicted_labels == 1) & (targets == 0) & (protected == 0)).float().sum()
        num_pred1_targets0_protected1 += ((predicted_labels == 1) & (targets == 0) & (protected == 1)).float().sum()
        num_pred0_targets1_protected0 += ((predicted_labels == 0) & (targets == 1) & (protected == 0)).float().sum()
        num_pred0_targets1_protected1 += ((predicted_labels == 0) & (targets == 1) & (protected == 1)).float().sum()
        num_pred1_targets1_protected0 += ((predicted_labels == 1) & (targets == 1) & (protected == 0)).float().sum()
        num_pred1_targets1_protected1 += ((predicted_labels == 1) & (targets == 1) & (protected == 1)).float().sum()

    # res imported from utils
    res = get_res_dict()

    res['acc'] = correct_pred.float()/num_examples * 100

    res['ddp'] = abs(num_correct_pred_protected0 / (num_protected0 + 1e-6) - num_correct_pred_protected1 / (num_protected1 + 1e-6)) * 100

    res['ppv'] = abs(num_pred1_targets1_protected0 / (num_pred1_protected0 + 1e-6) - num_pred1_targets1_protected1 / (num_pred1_protected1 + 1e-6)) * 100

    res['fpr'] = abs(num_pred0_targets0_protected0 / (num_targets0_protected0 + 1e-6) - num_pred0_targets0_protected1 / (num_targets0_protected1 + 1e-6)) * 100

    res['fnr'] = abs(num_pred1_targets1_protected0 / (num_targets1_protected0 + 1e-6) - num_pred1_targets1_protected1 / (num_targets1_protected1 + 1e-6)) * 100

    res['tn_s0'] = num_pred0_targets0_protected0
    res['tn_s1'] = num_pred0_targets0_protected1
    res['fp_s0'] = num_pred1_targets0_protected0
    res['fp_s1'] = num_pred1_targets0_protected1
    res['fn_s0'] = num_pred0_targets1_protected0
    res['fn_s1'] = num_pred0_targets1_protected1
    res['tp_s0'] = num_pred1_targets1_protected0
    res['tp_s1'] = num_pred1_targets1_protected1
    
    return res

In [105]:
def print_stats(config, epoch, stats, stat_type='Train'):
    if stat_type == 'Train':
        print('Epoch: %03d/%03d | Train Acc: %.3f%% | Train Ddp: %.3f%% | Train Ppv: %.3f%% | Train Fpr: %.3f%% | Train Fnr: %.3f%% ' % (
            epoch+1, config['NUM_EPOCHS'], 
            stats['acc'],
            stats['ddp'],
            stats['ppv'],
            stats['fpr'],
            stats['fnr']))
        print('                 | Train TP0: %d | Train FP0: %d | Train FN0: %d | Train TP0: %d' % (
            stats['tn_s0'],
            stats['fp_s0'],
            stats['fn_s0'],
            stats['tp_s0']))
        print('                 | Train TP1: %d | Train FP1: %d | Train FN1: %d | Train TP1: %d' % (
            stats['tn_s1'],
            stats['fp_s1'],
            stats['fn_s1'],
            stats['tp_s1']))
    elif stat_type == 'Valid' and not config['DEBUG']:
        print('Epoch: %03d/%03d | Valid Acc: %.3f%% | Valid Ddp: %.3f%% | Valid Ppv: %.3f%% | Valid Fpr: %.3f%% | Valid Fnr: %.3f%% ' % (
            epoch+1, config['NUM_EPOCHS'], 
            stats['acc'],
            stats['ddp'],
            stats['ppv'],
            stats['fpr'],
            stats['fnr']))
        print('                 | Valid TP0: %d | Valid FP0: %d | Valid FN0: %d | Valid TP0: %d' % (
            stats['tn_s0'],
            stats['fp_s0'],
            stats['fn_s0'],
            stats['tp_s0']))
        print('                 | Valid TP1: %d | Valid FP1: %d | Valid FN1: %d | Valid TP1: %d' % (
            stats['tn_s1'],
            stats['fp_s1'],
            stats['fn_s1'],
            stats['tp_s1']))
    elif stat_type == 'Test' and not config['DEBUG']:
        print('Epoch: %03d/%03d | Test Acc: %.3f%% | Test Ddp: %.3f%% | Test Ppv: %.3f%% | Test Fpr: %.3f%% | Test Fnr: %.3f%% ' % (
            epoch+1, config['NUM_EPOCHS'], 
            stats['acc'],
            stats['ddp'],
            stats['ppv'],
            stats['fpr'],
            stats['fnr']))
        print('                 | Test TP0: %d | Test FP0: %d | Test FN0: %d | Test TP0: %d' % (
            stats['tn_s0'],
            stats['fp_s0'],
            stats['fn_s0'],
            stats['tp_s0']))
        print('                 | Test TP1: %d | Test FP1: %d | Test FN1: %d | Test TP1: %d' % (
            stats['tn_s1'],
            stats['fp_s1'],
            stats['fn_s1'],
            stats['tp_s1']))

In [106]:
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))
with torch.set_grad_enabled(False): # save memory during inference
    test_stats = compute_accuracy(config, model, test_loader)
    print_stats(config, epoch, test_stats, stat_type='Test')

Total Training Time: 55.04 min
Epoch: 013/025 | Test Acc: 55.170% | Test Ddp: 31.852% | Test Ppv: 34.445% | Test Fpr: 1.894% | Test Fnr: 2.511% 
                 | Test TP0: 3964 | Test FP0: 299 | Test FN0: 6699 | Test TP0: 1285
                 | Test TP1: 5504 | Test FP1: 297 | Test FN1: 1654 | Test TP1: 260


The training log you provided seems to be related to a binary classification problem with metrics often used to assess both performance and fairness of a model. Let’s break down each component:

1. **Epoch**:  
   - `Epoch: 030/030` indicates that training has completed 30 out of 30 epochs.

2. **Test Accuracy (`Test Acc`)**:  
   - `Test Acc: 49.594%` refers to the overall accuracy of the model on the test set, meaning that approximately 49.59% of the test predictions were correct.

3. **Test Disparate Impact (`Test Ddp`)**:  
   - `Test Ddp: 40.357%` could represent a fairness metric, possibly indicating disparate impact or demographic disparity. It measures how different groups are treated in terms of positive predictions. A value closer to 100% would indicate less disparity.

4. **Test Positive Predictive Value (`Test Ppv`)**:  
   - `Test Ppv: 40.376%` indicates the precision, which is the proportion of true positive predictions out of all positive predictions. It measures how many predicted positives are actually positive.

5. **Test False Positive Rate (`Test Fpr`)**:  
   - `Test Fpr: 0.034%` shows the proportion of negatives that were incorrectly classified as positives. It's very low, indicating very few false positives relative to the total number of actual negatives.

6. **Test False Negative Rate (`Test Fnr`)**:  
   - `Test Fnr: 0.000%` indicates no false negatives, meaning all actual positive cases were correctly identified by the model.

7. **Confusion Matrix Breakdown**:  
   - `Test TP0: 7984`, `Test FP0: 4263`, `Test FN0: 0`, `Test TP1: 1914`, `Test FP1: 5799`, `Test FN1: 0`
   - These metrics provide the true positives (TP), false positives (FP), and false negatives (FN) for both classes (0 and 1):
     - `TP0` and `TP1`: True positives for class 0 and class 1, respectively.
     - `FP0` and `FP1`: False positives for class 0 and class 1.
     - `FN0` and `FN1`: False negatives for class 0 and class 1 (both are zero here).

Given that this is based on a fairness-oriented loss function, it seems like the metrics are aimed at evaluating not just the performance but also how well the model balances errors across different groups or classes. The fairness measures (like `Test Ddp`) are likely used to ensure that the model's predictions are equitable across groups. 

The results show a challenge in achieving high accuracy, indicating the model might still need tuning, especially in terms of improving both accuracy and fairness metrics simultaneously.

In [107]:
torch.save(model.state_dict(), '/kaggle/working/model_weights_30epochs.pth')

# Testing and compression

In [108]:
import torchvision.models as models


In [109]:


# Define your custom ResNet18 model
class CustomResNet18(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet18, self).__init__()
        self.base_model = models.resnet18(pretrained=False)  # No pretrained weights
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)  # Modify last layer

    def forward(self, x):
        return self.base_model(x)

# Initialize your custom model
num_classes = 2  # Replace with the number of classes in your dataset
model_download = CustomResNet18(num_classes)

# Load your custom weights
custom_weights_path = "/kaggle/input/resnet18_celeba_finetuned_pre_compression_/pytorch/default/1/model_weights_30epochs.pth"

# Load state_dict and handle missing or unexpected keys
state_dict = torch.load(custom_weights_path)
model_download.load_state_dict(state_dict, strict=False)  # Use strict=False to ignore mismatches

# Optionally, set the model to evaluation mode if you're using it for inference
model_download.eval()


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_30/3622632210.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

CustomResNet18(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [110]:
def model_size_evaluator(model_size_karo):
    num_params = sum(p.numel() for p in model_size_karo.parameters())
    size_mb = num_params * 4 / (1024 * 1024)  # assuming 4-byte floats
    print(f"Size of the model: {size_mb:.2f} MB")

In [111]:
model_size_evaluator(model_download)

Size of the model: 42.64 MB


# Experiments - Pruning followed by testing



**Pruning**

* Identify prunable parameters:
	+ Random: Prune random parameters.
	+ Magnitude: Prune parameters with smallest weights (L2 norm).
	+ Gradient: Prune based on accumulated gradient (requires backward pass).
	+ Information: Use high-order curvature information.
	+ Learned: Train network to prune itself (expensive).
* PyTorch supports random and magnitude-based pruning, which are effective and easy to compute.o compute.ny data.

![](https://miro.medium.com/v2/resize:fit:786/format:webp/1*l7Nx7TtTSHdqISSzbNrtiA.png)

Types of pruning - 
**Unstructured Pruning** 

Unstructured Puning refers to pruning individual atoms of parameters. E.g. individual weights in linear layers, individual filter pixels in convolution layers, some scaling floats in your custom layer, etc. The point is you prune parameters without their respective structure, hence the name unstructured pruning. - 

**Structured Pruning** 
As an alternative to unstructured pruning, structured pruning removes entire structures of parameters. This does not mean that it has to be an entire parameter, but you go beyond removing individual atoms e.g. in linear weights you’d drop entire rows or columns, or, in convolution layers entire filters (I point the interested reader to [1] where we have shown that many publicly available models contain a bunch of degenerated filters that should be prunable). In practice, you can achieve much higher pruning ratios with unstructured pruning, but it probably won’t speed up your model, as you still have to do all computations. Structured pruning can e.g. prune entire convolution channels and therefore significantly lower the number of matrix multiplications you need. Currently, there is a trend to support sparse tensors in both soft- and hardware, so in the future unstructured pruning may become highly relevant. 

**- Local vs. Global Pruning:** Pruning can happen per layer (local) or over all multiple/all layers (global). (global).

In [112]:
import torch.nn.utils.prune as prune

In [113]:
def test_function_forall(m, compute_accuracy = compute_accuracy, config=config, test_loader=test_loader):
    with torch.set_grad_enabled(False): # save memory during inference
        test_stats = compute_accuracy(config, m, test_loader)
        print_stats(config, 30, test_stats, stat_type='Test') #putting epoch as 0

In [114]:
model_download.cuda()
test_function_forall(model_download)

Epoch: 031/025 | Test Acc: 50.416% | Test Ddp: 40.383% | Test Ppv: 0.000% | Test Fpr: 0.000% | Test Fnr: 0.000% 
                 | Test TP0: 4263 | Test FP0: 0 | Test FN0: 7984 | Test TP0: 0
                 | Test TP1: 5801 | Test FP1: 0 | Test FN1: 1914 | Test TP1: 0


In [115]:
# Helper function to recursively add parameters to prune
def add_pruning_params(model, params_to_prune):
    for module in model.children():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            params_to_prune.append((module, 'weight'))
        else:
            # Recurse only if the module is not an atomic layer
            # This will correctly handle nested structures like nn.Sequential
            add_pruning_params(module, params_to_prune)

# Function to count the number of zero weights in a model
def count_zero_weights(model):
    num_zeros = 0
    num_total = 0
    for module in model.modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            num_zeros += torch.sum(module.weight == 0).item()
            num_total += module.weight.numel()
    return num_zeros, num_total

# 1. Global Structured Pruning (pruning entire channels across all layers)
def global_structured_pruning(model, amount=0.3):
    parameters_to_prune = []
    add_pruning_params(model, parameters_to_prune)
    prune.global_unstructured(parameters_to_prune, pruning_method=prune.LnStructured, amount=amount, n=2)
    return model

# 2. Global Unstructured Pruning (pruning individual weights across all layers)
def global_unstructured_pruning(model, amount=0.3):
    parameters_to_prune = []
    add_pruning_params(model, parameters_to_prune)
    prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=amount)
    return model

# 3. Local Structured Pruning (pruning channels in specific layers)
def local_structured_pruning(model, amount=0.5):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=0)
            prune.remove(module, 'weight')  # Apply the mask
    return model

# 4. Local Unstructured Pruning (pruning individual weights in specific layers)
def local_unstructured_pruning(model, amount=0.5):
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')  # Apply the mask
    return model

# # Apply each pruning technique and check results
# print("Applying Global Structured Pruning...")
# model_global_structured = global_structured_pruning(model, amount=0.3)
# num_zeros, num_total = count_zero_weights(model_global_structured)
# print(f"Global Structured Pruning: {num_zeros}/{num_total} weights are zero")

# print("Applying Global Unstructured Pruning...")
# model_global_unstructured = global_unstructured_pruning(model_download, amount=0.3)
# num_zeros, num_total = count_zero_weights(model_global_unstructured)
# print(f"Global Unstructured Pruning: {num_zeros}/{num_total} weights are zero")

# print("Applying Local Structured Pruning...")
# model_local_structured = local_structured_pruning(model_download, amount=0.5)
# num_zeros, num_total = count_zero_weights(model_local_structured)
# print(f"Local Structured Pruning: {num_zeros}/{num_total} weights are zero")

# print("Applying Local Unstructured Pruning...")
# model_local_unstructured = local_unstructured_pruning(model_download, amount=0.5)
# num_zeros, num_total = count_zero_weights(model_local_unstructured)
# print(f"Local Unstructured Pruning: {num_zeros}/{num_total} weights are zero")

In [116]:
import copy

# def test_function_forall(m, compute_accuracy, config, test_loader):
#     m.eval()  # Make sure the model is in evaluation mode
#     with torch.set_grad_enabled(False):  # Disable gradient calculations
#         test_stats = compute_accuracy(config, m, test_loader)
#         print_stats(config, 30, test_stats, stat_type='Test')  # Use 30 as the epoch (dummy value here)

# def test_function_forall(m, compute_accuracy = compute_accuracy, config=config, test_loader=test_loader):
#     with torch.set_grad_enabled(False): # save memory during inference
#         test_stats = compute_accuracy(config, m, test_loader)
#         print_stats(config, 30, test_stats, stat_type='Test') #putting epoch as 0

def prune_and_test_model(original_model, pruning_method, amount):
    # Step 1: Clone the original model to avoid modifying the original model
    pruned_model = copy.deepcopy(original_model)
    pruned_model.cuda()
    
    # Step 2: Apply the specified pruning method
    if pruning_method == 'global_structured':
        pruned_model = global_structured_pruning(pruned_model, amount)
    elif pruning_method == 'global_unstructured':
        pruned_model = global_unstructured_pruning(pruned_model, amount)
    elif pruning_method == 'local_structured':
        pruned_model = local_structured_pruning(pruned_model, amount)
    elif pruning_method == 'local_unstructured':
        pruned_model = local_unstructured_pruning(pruned_model, amount)
    else:
        raise ValueError("Invalid pruning method specified")

    # Step 3: Test the pruned model
    print(f"Testing {pruning_method} pruning...")
    model_size_evaluator(pruned_model)
    test_function_forall(m=pruned_model, compute_accuracy=compute_accuracy, config=config, test_loader=test_loader)

    # Step 4: Optionally, count the number of zero weights to verify pruning
    num_zeros, num_total = count_zero_weights(pruned_model)
    print(f"{pruning_method} Pruning: {num_zeros}/{num_total} weights are zero")

# Example usage for different pruning methods
# print("Testing Global Structured Pruning...")
# prune_and_test_model(model_download, 'global_structured', 0.3)

# print("Testing Global Unstructured Pruning...")
# prune_and_test_model(model_download, 'global_unstructured', 0.9)

# print("Testing Local Structured Pruning...")
# prune_and_test_model(model_download, 'local_structured', 0.5)

# print("Testing Local Unstructured Pruning...")
# prune_and_test_model(model_download, 'local_unstructured', 0.5)


In [117]:
def iterator_pruner(model, method):
    print(f"Testing {method}...\n")
    for i in range(1,11,1):
        # copy_model = copy.deepcopy(model)
        sparsity_number = i/10
        print(f"The sparsity is => {sparsity_number}")
        prune_and_test_model(model, method, sparsity_number)
        print()
    print("\n\n")

In [118]:
iterator_pruner(model_download, "global_unstructured")

Testing global_unstructured...

The sparsity is => 0.1
Testing global_unstructured pruning...
Size of the model: 42.64 MB
Epoch: 031/025 | Test Acc: 50.416% | Test Ddp: 40.383% | Test Ppv: 0.000% | Test Fpr: 0.000% | Test Fnr: 0.000% 
                 | Test TP0: 4263 | Test FP0: 0 | Test FN0: 7984 | Test TP0: 0
                 | Test TP1: 5801 | Test FP1: 0 | Test FN1: 1914 | Test TP1: 0
global_unstructured Pruning: 1116794/11167936 weights are zero

The sparsity is => 0.2
Testing global_unstructured pruning...
Size of the model: 42.64 MB
Epoch: 031/025 | Test Acc: 50.416% | Test Ddp: 40.383% | Test Ppv: 0.000% | Test Fpr: 0.000% | Test Fnr: 0.000% 
                 | Test TP0: 4263 | Test FP0: 0 | Test FN0: 7984 | Test TP0: 0
                 | Test TP1: 5801 | Test FP1: 0 | Test FN1: 1914 | Test TP1: 0
global_unstructured Pruning: 2233587/11167936 weights are zero

The sparsity is => 0.3
Testing global_unstructured pruning...
Size of the model: 42.64 MB
Epoch: 031/025 | Test Acc: 5

KeyboardInterrupt: 

In [ ]:
iterator_pruner(model_download, "local_structured")

In [ ]:
iterator_pruner(model_download, "local_unstructured")

In [ ]:
def count_zero_weights(model):
    num_zeros = 0
    num_total = 0
    for param in model.parameters():
        if param is not None:
            num_zeros += torch.sum(param == 0).item()
            num_total += param.numel()
    return num_zeros, num_total

# Before pruning
num_zeros_before, num_total_before = count_zero_weights(model_download)
print(f"Before Pruning: {num_zeros_before}/{num_total_before} weights are zero")

pruned_model = copy.deepcopy(model_download)
pruned_model = global_unstructured_pruning(pruned_model, 0.2)

# After pruning
num_zeros_after, num_total_after = count_zero_weights(pruned_model)
print(f"After Pruning: {num_zeros_after}/{num_total_after} weights are zero")


The provided code demonstrates different pruning techniques for a neural network model, which involve selectively removing some weights or entire structures (such as channels) to reduce the model’s size, potentially speed up inference, and mitigate overfitting while maintaining accuracy. Let’s explore these techniques and how they are implemented in code.

### Conceptual Overview of Pruning

1. **What is Pruning?**
   - Pruning is a model optimization technique used to eliminate unnecessary or less significant parameters (weights or structures) from a neural network. The goal is to simplify the model without significantly affecting its performance, which can result in a more compact model with faster inference times and lower memory usage.

2. **Types of Pruning in the Code:**
   - **Global Unstructured Pruning:** Removes individual weights across all layers based on a specified criterion, such as L1-norm, without considering the structure of the layers.
   - **Local Structured Pruning:** Eliminates entire structures, like channels or filters, within specific layers. This retains the network's architecture but reduces the number of computations per layer.
   - **Local Unstructured Pruning:** Prunes individual weights within specific layers, treating each layer independently and pruning based on some criteria (e.g., L1-norm).

### Code Explanation

#### 1. **Helper Function – `add_pruning_params`**

   - **Purpose:** This function traverses through the model's layers and adds the parameters that need to be pruned (in this case, weights of `nn.Conv2d` and `nn.Linear` layers) to a list.
   - **Implementation Details:**
     - It iterates over each module in the model using `model.children()`.
     - If a module is an instance of `nn.Conv2d` or `nn.Linear`, it appends a tuple containing the module and the string `'weight'` to the `params_to_prune` list. This tells the pruning methods that the weights of these layers should be considered for pruning.
     - If the module is not an atomic layer (like `nn.Sequential`), the function recursively calls itself to explore nested structures.

#### 2. **Global Unstructured Pruning – `global_unstructured_pruning`**

   - **Purpose:** Prunes a fraction of weights across all layers based on their importance, without considering the structure. The importance is determined using the L1 norm of the weights.
   - **Implementation Details:**
     - `parameters_to_prune` is initialized as an empty list.
     - The `add_pruning_params` function is called to populate `parameters_to_prune` with all the weights that can be pruned.
     - The `prune.global_unstructured` function applies pruning globally across the listed parameters, with `prune.L1Unstructured` as the pruning criterion. This method ranks the weights based on their L1 norm (absolute values) and removes the specified proportion (`amount=0.3` in this case).
     - The function returns the pruned model.

#### 3. **Local Structured Pruning – `local_structured_pruning`**

   - **Purpose:** Prunes entire structures, such as channels in convolutional layers or rows in linear layers. This retains the overall structure but reduces the layer's computational burden.
   - **Implementation Details:**
     - It iterates over each named module in the model using `model.named_modules()`.
     - If a module is an `nn.Conv2d` or `nn.Linear` layer, it applies `prune.ln_structured` to prune channels (or rows) based on the L2 norm (`n=2`). The `dim=0` argument specifies that the pruning is along the channel dimension (filters in Conv2D).
     - After pruning, `prune.remove` is called to finalize the pruning mask, updating the module's weight parameters accordingly. This step essentially removes the "pruning hooks" and makes the changes permanent.
     - The function returns the pruned model.

#### 4. **Local Unstructured Pruning – `local_unstructured_pruning`**

   - **Purpose:** Prunes individual weights within specific layers without considering the global distribution of weights.
   - **Implementation Details:**
     - Similar to local structured pruning, this function iterates over each named module.
     - For each `nn.Conv2d` or `nn.Linear` module, it applies `prune.l1_unstructured`, which prunes weights with the smallest absolute values (based on the L1 norm), retaining a specified percentage (`amount=0.5`).
     - `prune.remove` is used to finalize and apply the pruning, making the changes permanent.
     - The function then returns the modified model.

### How the Code Achieves Pruning

- **Parameter Selection:** The code identifies which weights or structures are eligible for pruning (using `add_pruning_params` and conditions for layer types).
- **Pruning Methods:**
  - `prune.global_unstructured` performs pruning across all eligible weights based on a global criterion.
  - `prune.ln_structured` and `prune.l1_unstructured` prune weights locally within individual layers, either by removing whole channels or individual weights.
- **Mask Application:** During pruning, masks are applied to the weights to zero out the pruned elements. These masks can be removed using `prune.remove`, which updates the model's parameters.

Each function essentially defines different pruning strategies, allowing flexibility in optimizing models based on various criteria. Do you need any specific part of the code or concept clarified further?

Sure! Let's simplify the explanation for **structured pruning**:

### What is Structured Pruning?

- Structured pruning removes entire "blocks" or "structures" within a layer, such as:
  - **Channels** in convolutional layers (like removing entire filters).
  - **Neurons** or **rows** in fully connected layers.
- Unlike unstructured pruning, which prunes individual weights, structured pruning prunes whole groups of weights together, effectively shrinking the layer's size.

### Meaning of `amount` in Structured Pruning

- In structured pruning, `amount` represents the **fraction of the structure** (e.g., channels in a convolutional layer) to prune.
- For example, if `amount=0.5`, it means **50% of the channels** in that layer will be removed.
- The code decides which channels to prune based on their importance, often measured by some norm (like L2 norm). Less important channels (those with smaller norm values) get pruned.

### Example for Clarity

Imagine a convolutional layer has **8 channels (filters)**:
- If `amount=0.5`, structured pruning will remove **4 of these channels**, effectively reducing the layer from 8 channels to 4.
- This makes the layer more compact and can speed up computations because fewer channels mean fewer operations.

In short, structured pruning shrinks layers by removing entire channels or neurons, and the `amount` specifies what percentage of them to prune away.

Great question! The reason we typically don’t use **structured global pruning** boils down to a few practical challenges and limitations:

### 1. **Complexity of Comparing Structures Across Different Layers**
   - In structured pruning, we are removing entire structures like channels (filters) or neurons, which are inherently different across layers. For example, comparing a channel in a convolutional layer to a row in a fully connected layer isn't straightforward because they have different shapes, functions, and impacts on the model.
   - Structured global pruning would require a way to rank and compare these different structures across all layers in a unified manner, which is much more complex than comparing individual weights.

### 2. **Maintaining Model Architecture Consistency**
   - Pruning entire channels or neurons affects the dimensionality of the input and output of layers. In structured local pruning, we can handle this within individual layers because we know how the inputs/outputs should change.
   - If we prune structures globally across layers, we may end up affecting connections between layers in unpredictable ways, potentially breaking the architecture or requiring significant re-engineering to adjust the dimensions.

### 3. **Layer-specific Importance**
   - Different layers in a neural network have different levels of importance. Early layers often capture basic features (e.g., edges in images), while deeper layers capture higher-level abstractions (e.g., objects).
   - Removing an entire structure (like a filter) from a deeper layer might have a more significant impact on performance compared to a shallow layer. Local pruning allows for layer-specific pruning criteria, ensuring that each layer is pruned in a way that accounts for its role in the network.

### In Contrast: Why Global Unstructured Pruning Works
- **Global unstructured pruning** works by comparing individual weights across all layers, which is simpler because every weight is a single scalar value. We can rank and prune these values regardless of their layer, making it suitable for global pruning.

### Summary
We don’t do **structured global pruning** because it’s difficult to compare and prune entire structures across different layers in a meaningful way while maintaining the model’s architecture. Local structured pruning is preferred since it allows for targeted pruning within specific layers, preserving the network's overall structure and functionality.